In [11]:
import pandas as pd
import numpy as np

from dataframes import get_pirep_df, get_util_df

In [12]:
df = get_util_df()

In [13]:
FH = df['FL_DURR'].sum()/np.timedelta64(1,'s')/3600


In [14]:
FH

370.81666666666666